In [170]:
import vgg16
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

In [171]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()

In [172]:
print(len(text))
print(text[:100])

600901
preface


supposing that truth is a woman--what then? is there not ground
for suspecting that all ph


In [14]:
!tail -n25 {path}

are thinkers who believe in the saints.


144

It stands to reason that this sketch of the saint, made upon the model
of the whole species, can be confronted with many opposing sketches that
would create a more agreeable impression. There are certain exceptions
among the species who distinguish themselves either by especial
gentleness or especial humanity, and perhaps by the strength of their
own personality. Others are in the highest degree fascinating because
certain of their delusions shed a particular glow over their whole
being, as is the case with the founder of christianity who took himself
for the only begotten son of God and hence felt himself sinless; so that
through his imagination--that should not be too harshly judged since the
whole of antiquity swarmed with sons of god--he attained the same goal,
the sense of complete sinlessness, complete irresponsibility, that can
now be attained by every individual through science.--In the same manner
I have viewed the saints of India

In [173]:
chars = sorted(list(set(text)))
chars.insert(0, '\0') # Make the index starting from 1
print(chars)
print(len(chars))
''.join(chars)

['\x00', '\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x86', '\xa4', '\xa6', '\xa9', '\xab', '\xc3']
60


'\x00\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyz\x86\xa4\xa6\xa9\xab\xc3'

In [174]:
char_to_index = {c:i for i, c in enumerate(chars)}
index_to_char = {i:c for i, c in enumerate(chars)}

In [175]:
indexes = [char_to_index[c] for c in text]

In [176]:
''.join([index_to_char[i] for i in indexes[:50]])

'preface\n\n\nsupposing that truth is a woman--what th'

In [177]:
max_len = 40
sentences = []
next_chars = []
for i in range(len(text)- max_len):
    sentences.append(text[i:i+max_len])
    next_chars.append(text[i+1:i+max_len+1])

In [178]:
print(len(sentences))
print(len(next_chars))
#print(text[:50])
print("%s\n%s"%(sentences[-1], next_chars[-1]))

600861
600861
n world as the indications of sinfulness
 world as the indications of sinfulness.


In [179]:
#[np.array(s) for s in sentences]
#m = np.concatenate([np.array([s for s in ss]) for ss in sentences])
sentences1 = np.row_stack([[s for s in ss] for ss in sentences])
next_chars1 = np.row_stack([[s for s in ss] for ss in next_chars])
print(sentences1.shape)
print(next_chars1.shape)

(600861, 40)
(600861, 40)


In [180]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [181]:
print(sentences.shape)
print(next_chars.shape)

(600859,)
(600859,)


In [152]:
np.array(sentences[0])

array('preface\n\n\nsupposing that truth is a woma',
      dtype='|S40')

In [124]:
print(nex_chars.shape)

(600861, 40)


In [195]:
sentences1[0]

array(['p', 'r', 'e', 'f', 'a', 'c', 'e', '\n', '\n', '\n', 's', 'u', 'p', 'p', 'o', 's', 'i', 'n',
       'g', ' ', 't', 'h', 'a', 't', ' ', 't', 'r', 'u', 't', 'h', ' ', 'i', 's', ' ', 'a', ' ',
       'w', 'o', 'm', 'a'],
      dtype='|S1')

In [184]:
n_fac = 24
vocab_size = len(chars)

In [185]:
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(320):
        x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [186]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=max_len),
        LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, recurrent_dropout=0.2, implementation=2, dropout=0.2, input_shape=(None, 24), return_sequences=True)`
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, recurrent_dropout=0.2, implementation=2, dropout=0.2, return_sequences=True)`


In [187]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [188]:
model.fit(sentences1, np.expand_dims(next_chars1,-1), batch_size=64, nb_epoch=1)

INFO (theano.gof.compilelock): Refreshing lock /home/ubuntu/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-2.7.12-64/lock_dir/lock


Epoch 1/1


ValueError: Bad input argument to theano function with name "train_function" at index 0 (0-based).  
Backtrace when that variable is created:

  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-186-83a304f41eaf>", line 10, in <module>
    Activation('softmax')
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/models.py", line 408, in __init__
    self.add(layer)
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/models.py", line 460, in add
    name=layer.name + '_input')
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/engine/topology.py", line 1439, in Input
    input_tensor=tensor)
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/legacy/interfaces.py", line 87, in wrapper
    return func(*args, **kwargs)
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/engine/topolog

In [ ]:
print_example()